# Face Recognition
### 1) Import libraries
### 2) Import video
### 3) Convert to b/w
### 4) Extract frames
### 5) Mark the region of face
### 6) Create Custom Dataset for each person
### 7) Create Train and test loaders
### 8) Create CNN Algorithm
### 9) Test and improve accuracy
### 9) Identify real time image

After we create the b/w images, then we will pass the images through an algorithm which will give the position of face in the image, then we will crop(resize) the image to that shape and we will fix a certain shape so as to pad these images and make every image of the same size. These images will be the input to Siamese Network.


In [2]:
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import OrderedDict

In [ ]:
person_name = OrderedDict()
person_name['Venkatesh'] = 1
person_name['Vivek'] = 2
person_name['Prince'] = 3
person_name['Abhay'] = 4

## Importing video and extracting frames

In [2]:
# video = cv2.VideoCapture('Sample.mp4')
# fps = video.get(cv2.CAP_PROP_FPS)
# print(fps)

# minutes = 0
# seconds = 5
# total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
# # seconds = total_frames//fps
# # minutes = seconds//60
# # seconds = seconds % 60
# # We can use os.chdir(path) to change the working directory
# arr = np.array([])
# width, height =  video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
# df = pd.DataFrame(columns = np.arange(int(width)*int(height)))
# for i in range(0,int(total_frames),10):
# #     frame_id = int(fps*(minutes*60 + seconds))
#     frame_id = i
# #     print('frame id =',frame_id)
#     video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
#     ret, frame = video.read()
# #     gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#     gray_image = frame
# #     print(type(frame))
# #     cv2.imshow('frame', frame); cv2.waitKey(0)
# #     cv2.imwrite(f'{frame_id}.png', frame)
#     img = np.asarray(gray_image)
# #     imgs = pd.DataFrame(img.reshape(1,-1))
# #     pd.concat([df,imgs])
# #     df.loc[i] = imgs
#     path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Venkatesh'
#     cv2.imwrite(os.path.join(path , f'{frame_id}.jpg'), gray_image)
# print("done")

## Crop the image till its face

In [3]:
# import cv2
# imagePath = 'sample.jpg'
# img = cv2.imread(imagePath) # This is already a numpy array
# gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# face_classifier = cv2.CascadeClassifier(
#     cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
# face = face_classifier.detectMultiScale(
#     gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
# for (x, y, w, h) in face:
#     cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)
# im_crop = gray_image[y:y+h, x:x+w] # Crop the image so that only face remains
# # im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2GRAY)

# # img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2RGB)
# import matplotlib.pyplot as plt
# plt.figure(figsize=(20,10))
# plt.imshow(im_crop)
# plt.axis('off')
# im_crop.shape

## Combining above two

In [59]:
# Function to extract images from video, and save just the faces from all the images in a seperate folder
def video_to_faces(video_path, label):
    video = cv2.VideoCapture(video_path)
    total_frames = video.get(cv2.CAP_PROP_FRAME_COUNT)
    # We can use os.chdir(path) to change the working directory
#     arr = np.array([])
    width, height =  video.get(cv2.CAP_PROP_FRAME_WIDTH), video.get(cv2.CAP_PROP_FRAME_HEIGHT)
    df = pd.DataFrame(columns = np.arange(int(width)*int(height)))
    face_classifier = cv2.CascadeClassifier(
        cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    max_h, max_w = 0,0
#     path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
    path = video_path.replace('Videos', 'Images')
    path = path.replace('.mp4', '')
#     print(path)
    os.makedirs(path, exist_ok=True)  # Ensure the directory exists or create it
    for i in range(0,int(total_frames),30):
        frame_id = i
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        ret, frame = video.read()
        gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        face = face_classifier.detectMultiScale(
            gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
        x, y, w, h = 0,0,0,0
#         print(face)
        if len(face) == 0: ## If no face is detected then it will skip this frame
            continue
        for (x, y, w, h) in face:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 4)
            x, y, w, h = x, y, w, h
        im_crop = gray_image[y:y+h, x:x+w]
#         im_crop = cv2.cvtColor(im_crop, cv2.COLOR_BGR2RGB)
        im_crop = cv2.cvtColor(im_crop, cv2.COLOR_GRAY2RGB)
        im_h, im_w, _ = im_crop.shape
        max_h, max_w = max(im_h, max_h), max(im_w, max_w) # Find the max shape of image
#         import matplotlib.pyplot as plt
#         plt.figure(figsize=(10,10))
#         plt.imshow(im_crop)
#         plt.axis('off')
#         path = 'C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\' + str(label)
        file_path = os.path.join(path , f'{frame_id}.jpg')
        if os.path.exists(file_path) :
            continue
        else:
            cv2.imwrite(file_path, im_crop)
    print("done")
    return max_h, max_w 
# max_h, max_w = video_to_faces('Abhay Singh.mp4', "Abhay singh")
# max_h, max_w = video_to_faces('C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos\\Abhay Singh.mp4', "Abhay Singh")
# print(max_h, max_w)
plt.close()

## Code to convert all the videos to face images and store them in seperate folders according to their names

In [60]:
video_dir = "C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Videos"
## Code to be used later to get a list of sub-folders in a folder
## folders = [x[0] for x in os.walk(video_dir)]
videos_list = [f for f in os.listdir(video_dir) if f.endswith(('.mp4'))]
working_dir = "C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Dataset\\Images"
max_list = []
for video in videos_list:
    print("Working")
    video_path = video_dir + "\\" + str(video)
    label = video.replace('.mp4','')
#     os.mkdir(label)
    max_h, max_w = video_to_faces(video_path, label)
#     print(max_h, max_w)
    max_list.append((max_h,max_w))
print("Done")

Working
done
Working
done
Working
done
Working
done
Working
done
Working
done
Working
done
Working


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


### Function to pad the image to a certain size

In [54]:
def pad(img, max_h, max_w):
    im_h, im_w, _ = img.shape
    left = (max_w - im_w)//2
    right = max_w - im_w - left
    top = (max_h - im_h)//2
    bottom = max_h - im_h - top 
    print(top,bottom, left, right)
    img_pad = np.pad(img, pad_width=((top,bottom), (left, right), (0, 0)))
    return img_pad
# print(max_h, max_w)
# img = cv2.imread("C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\Venkatesh\\330.jpg")
# print(img.shape)
# img_pad = pad(img, max_h, max_w)
# plt.imshow(img)
# plt.imshow(img_pad)

552 552
(660, 660, 3)
-54 -54 -54 -54


ValueError: index can't contain negative values

## Function to store the padded images in a dataset along with their label

In [1]:
# label, dataframe, label_dict
image_data = pd.DataFrame(columns = np.arange(int(max_h)*int(max_w)+1))
def pad_and_store(image_data, label,label_dict, max_h, max_w):
# Define the folder path where your images are located
    folder_path = "C:\\Users\\dhano\\Jupyter Notebooks\\Face Recognition\\" + label

# Get a list of all image files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg'))]
# Loop through the image files and open them
    for image_file in image_files:
        # Construct the full file path
        file_path = os.path.join(folder_path, image_file)
        image = cv2.imread(file_path)
        img_pad = pad(image, max_h, max_w)
        row = list(od[label])
        img_pad_flat = img_pad.reshape(1,-1)
        row.append(img_pad_flat)
        image_data.append(row)
    return image_data
image_data = pad_and_store(image_data, label,label_dict, max_h, max_w)

NameError: name 'pd' is not defined

# Code to Open Webcam, draw a rectangle around face and label it


In [29]:
# access video from the webcam
video_capture = cv2.VideoCapture(0)
face_classifier = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
def detect_bounding_box(vid):
    gray_image = cv2.cvtColor(vid, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray_image, 1.1, 5, minSize=(40, 40))
    x, y, w, h = 0,0,0,0
    for (x, y, w, h) in faces:
        cv2.rectangle(vid, (x, y), (x + w, y + h), (0, 255, 0), 4)
        x, y, w, h = x, y, w, h
    return faces, x, y
while True:
    result, video_frame = video_capture.read()  # read frames from the video
    if result is False:
        break  # terminate the loop if the frame is not read successfully
    faces, x, y = detect_bounding_box(
        video_frame
    )  # apply the function we created to the video frame
    title = 'Face'
    cv2.putText(video_frame,title , (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (30,255,30), 2)
    cv2.imshow(
        "My Face Detection Project", video_frame
    )
    # display the processed frame in a window named "My Face Detection Project"
    # add label to the face
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
video_capture.release()
cv2.destroyAllWindows()